In [33]:
showOutputs = True

def showTableau(c, a, b, certificate, operations, gap_vars):
    print("c[]: {}".format(c))
    print("A[] : {}".format(a))
    print("b[] : {}".format(b))

def identity(n):
    m=[[0 for x in range(n)] for y in range(n)]
    for i in range(0,n):
        m[i][i] = 1
    return m

def simplex_tableau(r, v, c, a, b):
    certificate = [0] * r
    operations = identity(3)
    gap_vars = identity(3)
    a = a + gap_vars
    c = c + [0] * r
    for i in range(len(c)):
        c[i] = c[i] * -1
    if(showOutputs): showTableau(c, a, b, certificate, operations, gap_vars)
        

In [34]:
r = 3
v = 3
c = [2,4,8]
a = [[1,0,0],[0,1,0],[0,0,1]]
b = [[1],[1],[1]]
simplex_tableau(r,v,c,a,b)

c[]: [-2, -4, -8, 0, 0, 0]
A[] : [[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 1, 0], [0, 0, 1]]
b[] : [[1], [1], [1]]
